# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.13it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [18]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# Rishwanth J  

📞 7305656635 | 📧 rishwanth738@gmail.com  
🔗 [LinkedIn](https://www.linkedin.com/in/rishwanth-j-9516a328a/) | [GitHub](https://github.com/Rishwanth738)  

---

## 📌 Professional Summary  
Highly motivated Computer Science student with strong foundations in data structures, algorithms, machine learning, and software design. Proficient in Python, C++, and JavaScript with hands-on experience developing AI-powered solutions and scalable backend systems. Demonstrated ability to build production-grade tools, fine-tune language models, and work across Linux-based environments. Adept at solving complex engineering problems related to NLP, distributed systems, and information retrieval.  

---

## 🎓 Education  

**Vellore Institute of Technology, Chennai**  
*BTech, Computer Science and Engineering*  
CGPA: **9.43/10** | Graduation: **Jun 2027**  

**Indian Institute of Technology Madras**  
*Bachelor of Science, Data Science*  
CGPA: **3.02/4** | Graduation: **Dec 2028**  

---

## 💼 Work Experience  

**Zoho Corporation** – Chennai, Tamil Nadu  
*Artificial Intelligence Intern* (Jun 2025 – Jul 2025)  
- Automated Postman script conversion from old format to newer format via fine-tuned LLMs.  
- Developed an MCP server and used prompt engineering to streamline QEngine script generation from a fine-tuned model on OpenUI Web.  

**Finari Private Services Limited** – Chennai, Tamil Nadu  
*AI & ML Intern* (May 2025 – Jun 2025)  
- Automated Email services and alerts at the company.  
- Streamlined company database acceptance criteria generation using fine-tuned LLMs and AWS services such as Polly.  

---

## 🏆 Leadership Experience  

**Nilgiri House, IIT Madras** – Chennai, Tamil Nadu  
*Regional Coordinator* (Nov 2024 – Jul 2025)  
- Coordinated and supported students enrolled in the IITM BS Degree program (Chennai region).  
- Managed events and resolved academic and administrative queries for students.  

---

## 🛠 Skills & Interests  

- **Languages & Tools:** Python, C, C++, Java, HTML, CSS, JavaScript, R, MySQL, PostgreSQL, React JS, Node JS, Git, GitHub  
- **Specializations:** NLP, Generative AI  

---

## 🚀 Projects  

**[SmartLogiX: AI-Powered Logistics Platform](https://github.com/Rishwanth738/SmartLogiX-An-AI-Powered-Logistics-Optimization-Platform)**  
- Developed an AI-powered logistics optimization platform integrating RL-based routing, predictive forecasting, and automated dispatch systems.  
- Achieved **24% reduction** in operational costs and reduced software expenses by **90%** via open-source APIs (TomTom, OSRM, Weatherbit, AQICN).  

**[Freshness & OCR Detection – Factory Oriented](https://github.com/Rishwanth738/Freshness-and-ocr-detection-Factory-oriented)**  
- Achieved **<1000ms real-time brand recognition** with 95%+ accuracy.  
- Implemented expiry date extraction using regex with **98%+ precision**.  
- Designed freshness evaluation model trained on real-world samples.  

**[Suicide Prevention Hotline App](https://github.com/Rishwanth738/HopeLine---Suicide-Prevention-Hotline-app)**  
- Built AI-powered hotline for real-time crisis intervention using socket programming.  
- Integrated chatbot with LLM fine-tuned on therapy conversations for immediate guidance and support.  

---

## 🏅 Hackathons & Certifications  

**Hackathons:**  
- Flipkart Grid 6.0 Robotics Challenge – *Semifinalists*  
- Innovate 4.0 – *Runners Up*  
- Innovate Carlo Rino UPM UI/UX – *Semifinalists*  
- SCOPE Champ Expo – *Winners*  

**Certifications:**  
- IITM Data Science BS Degree (Foundational Level) – *IIT Madras, 2024*  
- Generative AI – *Oracle, 2024*  
- Intro to Gen AI with AWS – *Udacity, 2024*  
- Machine Learning and Deep Learning – *Udemy, 2024*  
- Python – *IIT Bombay Spoken Tutorial, 2024*  
- C and C++ – *IIT Bombay Spoken Tutorial, 2024*  


## Creating Agents

In [23]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [24]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [25]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [26]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [27]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [28]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [29]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [30]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [31]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

2025-09-04 08:09:04,344 - 140585060408192 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [37]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/4293869991/?alternateChannel=search&eBP=CwEAAAGZE8yRlZskp88ughCiuNyiDZgGLOFFd3akOUxHhX-1GD6E4tVW6s0AV4ZQS1t-oAZguLnQaEJVCBv4-HlAkcUUrpq83aqOVxtf7CJvDcZUTlOX_AINKTY9J9UOzPwKF3au9GqNe2vZOggGxphlKpIhx4lFBt4BX5fEnjRSWpNk5ktJ_ZN4RuK5OwnGiSppMGx1ab2wEbGCmQRy_RlYv8S9K-20jScQA4COZ_O-PJfDP2u8j1U2hmOAzfuz8ROjbvaWwYSqQopXzzsJSIkXb02pBxuKlDs-4Pf07qcDwTIuka3NdiVMkt95_cbv7g9-TN5R2NvFFMM8x2jAgZvXYvihQVnnECh0peceHa6AHVp5dvQC41aGYjGKEbwXiGH0OARxRlkKsdjIl0XiezkTC8o-ylk6VVhsXh1wSbFqOjkI6lm83Km_uYads_iPABhSBXSo_eM2qhone0xbJ7jVX9yxQklS-MKqHn6Waxrhm4yi9XVjVxw&refId=6X%2B0RQvVhAmE%2Bj2U3gqcww%3D%3D&trackingId=4Wn3eoMrmNcOSLt%2BxPoHGg%3D%3D',
    'github_url': 'https://github.com/Rishwanth738',
    'personal_writeup': """Rishwanth J, a
    highly motivated Computer Science student with strong foundations in data structures, algorithms, machine learning,
and software design. Proficient in Python, C++, and JavaScript with hands-on experience developing AI-powered
solutions and scalable backend systems. Demonstrated ability to build production-grade tools, fine-tune language
models, and work across Linux-based environments. Adept at solving complex engineering problems related to NLP,
distributed systems, and information retrieval."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [41]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/4293869991/?alternateChannel=search&eBP=CwEAAAGZE8yRlZskp88ughCiuNyiDZgGLOFFd3akOUxHhX-1GD6E4tVW6s0AV4ZQS1t-oAZguLnQaEJVCBv4-HlAkcUUrpq83aqOVxtf7CJvDcZUTlOX_AINKTY9J9UOzPwKF3au9GqNe2vZOggGxphlKpIhx4lFBt4BX5fEnjRSWpNk5ktJ_ZN4RuK5OwnGiSppMGx1ab2wEbGCmQRy_RlYv8S9K-20jScQA4COZ_O-PJfDP2u8j1U2hmOAzfuz8ROjbvaWwYSqQopXzzsJSIkXb02pBxuKlDs-4Pf07qcDwTIuka3NdiVMkt95_cbv7g9-TN5R2NvFFMM8x2jAgZvXYvihQVnnECh0peceHa6AHVp5dvQC41aGYjGKEbwXiGH0OARxRlkKsdjIl0XiezkTC8o-ylk6VVhsXh1wSbFqOjkI6lm83Km_uYads_iPABhSBXSo_eM2qhone0xbJ7jVX9yxQklS-MKqHn6Waxrhm4yi9XVjVxw&refId=6X%2B0RQvVhAmE%2Bj2U3gqcww%3D%3D&trackingId=4Wn3eoMrmNcOSLt%2BxPoHGg%3D%3D) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Pers

Action: Read website content
Action Input: {"website_url": "https://github.com/Rishwanth738"} 

Rishwanth738 (Rishwanth J) · GitHub
Skip to content
Navigation Menu
Toggle navigation
 Sign in
Appearance settings
 Platform
 GitHub Copilot
 Write better code with AI
 GitHub Spark
 New
 Build and deploy intelligent apps
 GitHub Models
 New
 Manage and compare prompts
 GitHub Advanced Security
 Find and fix vulnerabilities
 Actions
 Automate any workflow
 Codespaces
 Instant dev environments
 Issues
 Plan and track work
 Code Review
 Manage code changes
 Discussions
 Collaborate outside of code
 Code Search
 Find more, search less
Explore
 Why GitHub
 Documentation
 GitHub Skills
 Blog
Integrations
 GitHub Marketplace
 View all features
 Solutions
By company size
 Enterprises
 Small and medium teams
 Startups
 Nonprofits
By use case
 DevSecOps
 DevOps
 CI/CD
 View all use cases
By industry
 Healthcare
 Financial services
 Manufacturing
 Government
 View all industries
 View all solutions
 R

Action: Read website content
Action Input: {"website_url": "https://github.com/Rishwanth738"} 

Rishwanth738 (Rishwanth J) · GitHub
Skip to content
Navigation Menu
Toggle navigation
 Sign in
Appearance settings
 Platform
 GitHub Copilot
 Write better code with AI
 GitHub Spark
 New
 Build and deploy intelligent apps
 GitHub Models
 New
 Manage and compare prompts
 GitHub Advanced Security
 Find and fix vulnerabilities
 Actions
 Automate any workflow
 Codespaces
 Instant dev environments
 Issues
 Plan and track work
 Code Review
 Manage code changes
 Discussions
 Collaborate outside of code
 Code Search
 Find more, search less
Explore
 Why GitHub
 Documentation
 GitHub Skills
 Blog
Integrations
 GitHub Marketplace
 View all features
 Solutions
By company size
 Enterprises
 Small and medium teams
 Startups
 Nonprofits
By use case
 DevSecOps
 DevOps
 CI/CD
 View all use cases
By industry
 Healthcare
 Financial services
 Manufacturing
 Government
 View all industries
 View all solutions
 R

Thought: I need to extract the key skills, experiences, and qualifications required for the Data Science Intern position at Recruit Forge from the job posting.

Action: Read website content
Action Input: {"website_url": "https://www.linkedin.com/jobs/view/4293869991/?alternateChannel=search&eBP=CwEAAAGZE8yRlZskp88ughCiuNyiDZgGLOFFd3akOUxHhX-1GD6E4tVW6s0AV4ZQS1t-oAZguLnQaEJVCBv4-HlAkcUUrpq83aqOVxtf7CJvDcZUTlOX_AINKTY9J9UOzPwKF3au9GqNe2vZOggGxphlKpIhx4lFBt4BX5fEnjRSWpNk5ktJ_ZN4RuK5OwnGiSppMGx1ab2wEbGCmQRy_RlYv8S9K-20jScQA4COZ_O-PJfDP2u8j1U2hmOAzfuz8ROjbvaWwYSqQopXzzsJSIkXb02pBxuKlDs-4Pf07qcDwTIuka3NdiVMkt95_cbv7g9-TN5R2NvFFMM8x2jAgZvXYvihQVnnECh0peceHa6AHVp5dvQC41aGYjGKEbwXiGH0OARxRlkKsdjIl0XiezkTC8o-ylk6VVhsXh1wSbFqOjkI6lm83Km_uYads_iPABhSBXSo_eM2qhone0xbJ7jVX9yxQklS-MKqHn6Waxrhm4yi9XVjVxw&refId=6X%2B0RQvVhAmE%2Bj2U3gqcww%3D%3D&trackingId=4Wn3eoMrmNcOSLt%2BxPoHGg%3D%3D"} 

Recruit Forge hiring Data Science Intern in India | LinkedIn
 Skip to main content
LinkedIn
 Data Science Intern in

Thought: I have gathered information from Rishwanth J's GitHub profile and personal write-up. Now, I need to synthesize this information to create a detailed personal and professional profile for him.

Final Answer:
Rishwanth J is a highly motivated Computer Science student with a strong foundation in data structures, algorithms, machine learning, and software design. He is proficient in Python, C++, and JavaScript with hands-on experience developing AI-powered solutions and scalable backend systems. Rishwanth has demonstrated the ability to build production-grade tools, fine-tune language models, and work across Linux-based environments. He is adept at solving complex engineering problems related to NLP, distributed systems, and information retrieval. Rishwanth is currently working as an Intern at Zoho and is dedicated to learning Node.js, GANs, and NLP. He is passionate about Data Science, AI, Web Development, and anything cool involving code. Additionally, Rishwanth loves cricket an

Thought: 
Now that I have gathered the key responsibilities for the Data Science Intern position and reviewed Rishwanth J's current resume, I need to update his resume to better align with the job requirements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Personal Profiler for Engineers", "task": "Update resume with relevant job responsibilities and qualifications", "context": "Update Rishwanth J's resume to highlight his qualifications and experiences in line with the key responsibilities and qualifications for the Data Science Intern position (Job ID: RF387)."}

> Entering new CrewAgentExecutor chain...
I need to gather information related to Rishwanth J's qualifications and experiences for the Data Science Intern position (Job ID: RF387) in order to update his resume effectively.

Action: Read a file's content
Action Input: {} 

# Rishwanth J  

📞 7305656635 | 📧 rishwanth738@gmail.com  
🔗 [LinkedIn](https://www.linkedin.com/in/rishwanth-j-9516a328a/) | [GitHub](ht

Final Answer:
# Rishwanth J  

📞 7305656635 | 📧 rishwanth738@gmail.com  
🔗 [LinkedIn](https://www.linkedin.com/in/rishwanth-j-9516a328a/) | [GitHub](https://github.com/Rishwanth738)  

---

## 📌 Professional Summary  
Highly motivated Computer Science student with a strong foundation in data structures, algorithms, machine learning, and software design. Proficient in Python, C++, and JavaScript, with hands-on experience developing AI-powered solutions and scalable backend systems. Skilled in building production-grade tools, fine-tuning language models, and working across Linux-based environments. Adept at solving complex engineering problems related to NLP, distributed systems, and information retrieval.

---

## 🎓 Education  

**Vellore Institute of Technology, Chennai**  
*BTech, Computer Science and Engineering*  
CGPA: **9.43/10** | Graduation: **Jun 2027**  

**Indian Institute of Technology Madras**  
*Bachelor of Science, Data Science*  
CGPA: **3.02/4** | Graduation: **Dec 2028

Thought: I need to gather more information to create relevant interview questions and talking points for the Data Science Intern position based on Rishwanth J's resume.

Action: Search a MDX's content
Action Input: {} 

I encountered an error while trying to use the tool. This was the error: MDXSearchTool._run() missing 1 required positional argument: 'search_query'.
 Tool Search a MDX's content accepts these inputs: A tool that can be used to semantic search a query the ./fake_resume.md MDX's content.

Thought: I need to gather more information to create relevant interview questions and talking points for the Data Science Intern position based on Rishwanth J's resume.

Action: Read a file's content
Action Input: {} 

# Rishwanth J  

📞 7305656635 | 📧 rishwanth738@gmail.com  
🔗 [LinkedIn](https://www.linkedin.com/in/rishwanth-j-9516a328a/) | [GitHub](https://github.com/Rishwanth738)  

---

## 📌 Professional Summary  
Highly motivated Computer Science student with strong foundations in

Thought: I have gathered all the necessary information from Rishwanth J's resume and the job requirements to create relevant interview questions and talking points.

Final Answer:
### Interview Questions and Talking Points for Data Science Intern Position:

1. **Technical Skills Assessment:**
   - Discuss your experience with Python and how you have used libraries like Pandas, NumPy, and Scikit-learn in your projects.
   - Can you explain a project where you applied machine learning techniques such as regularization, boosting, random forests, ensemble methods, or time series modeling?
   - Share details about your exposure to LLMs and Generative AI. How have you implemented these concepts in your projects?

2. **Project Experience:**
   - Walk us through the SmartLogiX project. How did you achieve a 24% reduction in operational costs, and how did you integrate open-source APIs for efficiency?
   - Explain the Freshness & OCR Detection project. How did you ensure real-time brand recogni

- Dislplay the generated `tailored_resume.md` file.

In [42]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Rishwanth J  

📞 7305656635 | 📧 rishwanth738@gmail.com  
🔗 [LinkedIn](https://www.linkedin.com/in/rishwanth-j-9516a328a/) | [GitHub](https://github.com/Rishwanth738)  

---

## 📌 Professional Summary  
Highly motivated Computer Science student with a strong foundation in data structures, algorithms, machine learning, and software design. Proficient in Python, C++, and JavaScript, with hands-on experience developing AI-powered solutions and scalable backend systems. Skilled in building production-grade tools, fine-tuning language models, and working across Linux-based environments. Adept at solving complex engineering problems related to NLP, distributed systems, and information retrieval.

---

## 🎓 Education  

**Vellore Institute of Technology, Chennai**  
*BTech, Computer Science and Engineering*  
CGPA: **9.43/10** | Graduation: **Jun 2027**  

**Indian Institute of Technology Madras**  
*Bachelor of Science, Data Science*  
CGPA: **3.02/4** | Graduation: **Dec 2028**  

---

## 💼 Work Experience  

**Zoho Corporation** – Chennai, Tamil Nadu  
*Artificial Intelligence Intern* (Jun 2025 – Jul 2025)  
- Automated Postman script conversion using fine-tuned LLMs.  
- Developed an MCP server to streamline QEngine script generation.

**Finari Private Services Limited** – Chennai, Tamil Nadu  
*AI & ML Intern* (May 2025 – Jun 2025)  
- Automated Email services and alerts.  
- Streamlined company database acceptance criteria generation using fine-tuned LLMs and AWS services.

---

## 🏆 Leadership Experience  

**Nilgiri House, IIT Madras** – Chennai, Tamil Nadu  
*Regional Coordinator* (Nov 2024 – Jul 2025)  
- Coordinated and supported students in the IITM BS Degree program.  
- Managed events and resolved academic and administrative queries.

---

## 🛠 Skills & Interests  

- **Languages & Tools:** Python, C, C++, Java, HTML, CSS, JavaScript, R, MySQL, PostgreSQL, React JS, Node JS, Git, GitHub  
- **Specializations:** NLP, Generative AI  

---

## 🚀 Projects  

**[SmartLogiX: AI-Powered Logistics Platform](https://github.com/Rishwanth738/SmartLogiX-An-AI-Powered-Logistics-Optimization-Platform)**  
- Developed an AI-powered logistics optimization platform with significant cost reduction and enhanced efficiency.

**[Freshness & OCR Detection – Factory Oriented](https://github.com/Rishwanth738/Freshness-and-ocr-detection-Factory-oriented)**  
- Real-time brand recognition and expiry date extraction with high accuracy.

**[Suicide Prevention Hotline App](https://github.com/Rishwanth738/HopeLine---Suicide-Prevention-Hotline-app)**  
- Built an AI-powered hotline for crisis intervention and integrated chatbot with LLM fine-tuned on therapy conversations.

---

## 🏅 Hackathons & Certifications  

**Hackathons:**  
- Flipkart Grid 6.0 Robotics Challenge – *Semifinalists*  
- Innovate 4.0 – *Runners Up*  
- Innovate Carlo Rino UPM UI/UX – *Semifinalists*  
- SCOPE Champ Expo – *Winners*  

**Certifications:**  
- IITM Data Science BS Degree (Foundational Level) – *IIT Madras, 2024*  
- Generative AI – *Oracle, 2024*  
- Intro to Gen AI with AWS – *Udacity, 2024*  
- Machine Learning and Deep Learning – *Udemy, 2024*  
- Python – *IIT Bombay Spoken Tutorial, 2024*  
- C and C++ – *IIT Bombay Spoken Tutorial, 2024*

- Dislplay the generated `interview_materials.md` file.

In [43]:
display(Markdown("./interview_materials.md"))


### Interview Questions and Talking Points for Data Science Intern Position:

1. **Technical Skills Assessment:**
   - Discuss your experience with Python and how you have used libraries like Pandas, NumPy, and Scikit-learn in your projects.
   - Can you explain a project where you applied machine learning techniques such as regularization, boosting, random forests, ensemble methods, or time series modeling?
   - Share details about your exposure to LLMs and Generative AI. How have you implemented these concepts in your projects?

2. **Project Experience:**
   - Walk us through the SmartLogiX project. How did you achieve a 24% reduction in operational costs, and how did you integrate open-source APIs for efficiency?
   - Explain the Freshness & OCR Detection project. How did you ensure real-time brand recognition and high accuracy in expiry date extraction?
   - Tell us about the Suicide Prevention Hotline App. How did you utilize AI for crisis intervention and integrate a chatbot fine-tuned on therapy conversations?

3. **Problem-Solving and Collaboration:**
   - Describe a challenging problem you encountered in one of your projects. How did you approach and solve it?
   - How do you effectively communicate data insights to non-technical team members or stakeholders?
   - Share an experience where you collaborated with a team to deliver a successful project. What was your role, and how did you ensure effective teamwork?

4. **Soft Skills and Personal Development:**
   - How do you stay updated with the latest trends and advancements in Data Science and AI?
   - Discuss a time when you had to adapt to new tools or technologies quickly. How did you manage the learning curve?
   - In what ways do you think your leadership experience as a Regional Coordinator has contributed to your professional development and teamwork skills?

5. **Future Goals and Learning Opportunities:**
   - What are your career aspirations in the field of Data Science and AI? How do you plan to achieve them?
   - How do you see yourself contributing to a team focused on AdTech optimization, customer analytics, and generative AI?
   - Are there any specific areas within Data Science or AI that you are keen on exploring further, and why?

6. **Role-Specific Situational Questions:**
   - Given a scenario where you need to develop a customer analytics model, what steps would you take to ensure its accuracy and relevance to business objectives?
   - How would you approach a project involving Generative AI to create innovative solutions for a business problem?
   - If tasked with continuous improvement of an existing machine learning algorithm, what strategies would you implement to enhance its performance?

These interview questions and talking points are designed to help you showcase your skills, experiences, and capabilities effectively during the interview process. Good luck!

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)